# Robust methods for intervention selections using Thompson Sampling
This notebooks explores robust methods for using thompson sampling in Self-E

In [ ]:
# Imports & preperation code
%load_ext autoreload
%autoreload 2

import numpy
from adaptive_nof1.models import Model, SelfExperimentationModel
from adaptive_nof1.basic_types import Outcome, History

from adaptive_nof1 import SeriesOfSimulationsRunner, SeriesOfSimulationsData
from adaptive_nof1.policies import (
    FixedPolicy,
    ThompsonSampling,
    BalancedThompsonSampling,
    ClippedThompsonSampling,
    ComposedPolicy,
)
from adaptive_nof1.inference import *
from adaptive_nof1.helpers import *
from adaptive_nof1.series_of_simulations_runner import simulate_configurations
from adaptive_nof1.metrics import SimpleRegret, RegretAgainstOtherConfiguration
from sklearn.linear_model import Ridge
import matplotlib.pyplot as plt
import holoviews
import pandas

In [ ]:
## Helper functions to do the analyses
def switch_config(config):
    config = config.copy()
    config["policy"] = "FixedPolicy"
    return config


def plot_regret_against_fixed(calculated_series, config_to_simulation_data):
    models = numpy.unique([s["configuration"]["model"] for s in calculated_series])
    for model in models:
        plt.figure()
        plt.title(model)
        SeriesOfSimulationsData.plot_lines(
            [
                series["result"]
                for series in calculated_series
                if series["configuration"]["model"] == model
            ],
            [
                RegretAgainstOtherConfiguration(
                    configuration_transform_function=switch_config,
                    config_to_simulation_data=config_to_simulation_data,
                    outcome_name="outcome",
                )
            ],
            legend_position=(0, 1.5),
        )

## Methodology

We start with a simple simulation model, and then add different features with make learning harder, namely:
- changes in baseline
- correlation in the outcomes
- spikes in the data.
After analysing how the algorithms behave, we will test different additions to it.
## Simulation Model
We assume ordinal outcomes between 0 and 5 for our simulation model.
It is assumed that this ordinal measure is achieved by mapping a continuous outcome onto 5 point scale. The decision boundaries are chosen by dividing the probability mass of a $N(0,1)$ distribution into equal parts. They are -0.84, -0.25, 0.25 and 0.84.
In the most basic case, the outcome is defined as: $outcome(a) = baseline + \text{effect}(a)$.

## Policies
We start with three policies for comparison:
- Fixed, Balanced Design
- Thompson Sampling with Beta Model
- Thompson Sampling with Conjugate Normal Model

## Bayesian Models
### BetaModel
This is currently used in the Self-E app.
Let $c$ be the number of the condition.
This model estimates the probability $p$ of an intervention beeing sucessfull as $p \sim Beta(a_c + 1, b_c + 1)$, where $a = count(outcome(c) \geq average(outcome))$ "number of sucesses", and $b = count(outcome(c) < average(outcome)$ "number of failures".
Then, it uses sampling to estimate the probability $Beta(a_1,b_1) > Beta(a_2, b_2)$
Since $a_c$ and $b_c$ are 0 if there is no data $Beta(1, 1)$ it the prior distribution.

### ConjugateNormalModel
This model models each outcome as a Normal random variable $N(\mu_c,\sigma_c)$.  
It uses the normal inverse gamma function $ING(mean, l, alpha, beta)$ as prior, because this is the conjugate prior to the normal distribution and therefore makes updating very easy.
The used priors are shown below, and are set to be weakly informative. The prior predictive simulation shows how the predicted values range from 0 to 5.


## Evaluation
We evaluate different metrics that could be interesting:  
Regret is defined as the difference in the sum of outcomes between to policies. For our case, we want to compare against the fixed schedule.
We define the regret as  
$Regret(policy) = \sum_{t}^T Reward(t)_{fixed} - \sum_{t}^T Reward(t)_{adaptive}$.

We evaluate:
- Mean Regret
- Median Regret
- 0.25 and 0.75 quantiles of regret
- Span between best and worst regret (per participant)
- Best Intervention Identification: Percentage of Participants getting the correct prediction at timepoint t
  - For fixed, this will be determined by difference in means between both conditions
  - For the Bayesian Models, the preferred condition will be determined by the maximum probability of selection at the last timepoint


In [ ]:
# Code for setting priors
conjugate_normal_model_priors = {
    "mean": 2.5,
    "l": 0.5,
    "alpha": 10,
    "beta": 10,
}
## Print posterior distribution of priors
model = ConjugateNormalModel(**conjugate_normal_model_priors)
plt.hist(
    model.sample_normal_inverse_gamma(
        model.mean, model.l, model.alpha, model.beta, 10000, 1
    ),
    bins=100,
)
plt.title("Prior predictive distribution for ConjugateNormalModel")

In [ ]:
# Code for calculating evaluation
def calculate_recommendations(calculated_series):
    data = []
    for result in calculated_series:
        series_of_simulations_data = result["result"]
        for simulation in series_of_simulations_data.simulations:
            observation = simulation.history.observations[-1]
            if "probabilities" in observation.debug_data:
                # print(observation.debug_data["probabilities"])
                probabilities = observation.debug_data["probabilities"]
                data.append(
                    {
                        **simulation.configuration,
                        "p_assigned_intervention_1": probabilities[1],
                        "p_assigned_intervention_0": probabilities[0],
                        "recommendation": numpy.argmax(probabilities),
                    }
                )
                # print(data)
            else:
                df = simulation.history.to_df()
                data.append(
                    {
                        **simulation.configuration,
                        "recommendation": numpy.argmax(
                            df.groupby("treatment")["outcome"].mean()
                        ),
                    }
                )
    recommended_interventions_df = pandas.DataFrame(data)
    return recommended_interventions_df


def generate_result_table(calculated_series):
    # Things for the table: mean regret, median regret, 25% regret, 75% regret, bias when fitting linear regression model
    metrics = [
        RegretAgainstOtherConfiguration(
            configuration_transform_function=switch_config,
            config_to_simulation_data=config_to_simulation_data,
            outcome_name="outcome",
        ),
    ]
    series = [entry["result"] for entry in calculated_series]
    scores = SeriesOfSimulationsData.score_data(
        series,
        metrics,
    )
    scores = scores[scores["t"] == scores["t"].max()]
    table = pandas.DataFrame()
    table["mean_regret"] = scores.groupby(["policy", "model"])["score"].mean()
    table["median_regret"] = scores.groupby(["policy", "model"])["score"].median()
    table[".25 quantile regret"] = scores.groupby(["policy", "model"])[
        "score"
    ].quantile(0.25)
    table[".75 quantile regret"] = scores.groupby(["policy", "model"])[
        "score"
    ].quantile(0.75)
    table[".95 quantile regret"] = scores.groupby(["policy", "model"])[
        "score"
    ].quantile(0.95)
    table[".05 quantile regret"] = scores.groupby(["policy", "model"])[
        "score"
    ].quantile(0.05)
    table["Span of regret"] = (
        scores.groupby(["policy", "model"])["score"].max()
        - scores.groupby(["policy", "model"])["score"].min()
    )
    recommended_interventions_df = calculate_recommendations(calculated_series)
    table["p_assigned_intervention_1_mean"] = recommended_interventions_df.groupby(
        ["policy", "model"]
    )["p_assigned_intervention_1"].mean()
    table["recommendation_mean"] = recommended_interventions_df.groupby(
        ["policy", "model"]
    )["recommendation"].mean()
    return table.swaplevel("policy", "model").sort_index()

In [ ]:
# Prepare simulation constants
N_INTERVENTIONS = 2
LENGTH = 40
N_PATIENTS = 100

## Scenario 1:
Let's start with a simple scenario: $baseline \sim Normal(0, 1)$, Treatment effects differ.
We look at 3 different effects:
- Scenario 1.1: -1 // Treatment worsens outcome
- Scenario 1.2: 0  // Treatment does not change outcome
- Scenario 1.3: 1  // Treatment improves outcome

In [ ]:
# Define models
data_generating_model_1_1 = lambda patient_id: SelfExperimentationModel(
    patient_id,
    intervention_effects=[0, -1],
    baseline_model="noise",
    baseline_config={"variance": 1},
)
data_generating_model_1_2 = lambda patient_id: SelfExperimentationModel(
    patient_id,
    intervention_effects=[0, 0],
    baseline_model="noise",
    baseline_config={"variance": 1},
)
data_generating_model_1_3 = lambda patient_id: SelfExperimentationModel(
    patient_id,
    intervention_effects=[0, 1],
    baseline_model="noise",
    baseline_config={"variance": 1},
)

In [ ]:
# Define policies
fixed_policy = FixedPolicy(number_of_actions=N_INTERVENTIONS)
linear_ts = ThompsonSampling(
    inference_model=ConjugateNormalModel(**conjugate_normal_model_priors),
    number_of_actions=N_INTERVENTIONS,
)
beta_ts = ThompsonSampling(
    inference_model=BetaModel(),
    number_of_actions=N_INTERVENTIONS,
)
regression_ts = ThompsonSampling(
    inference_model=BayesianLinearRegressionModel(
        mean=numpy.array(
            [
                conjugate_normal_model_priors["mean"],
                conjugate_normal_model_priors["mean"],
            ]
        ),
        v=numpy.eye(N_INTERVENTIONS) * 100,
        alpha=conjugate_normal_model_priors["alpha"],
        beta=conjugate_normal_model_priors["beta"],
    ),
    number_of_actions=N_INTERVENTIONS,
)

In [ ]:
# Full crossover study
study_designs = {
    "n_patients": [N_PATIENTS],
    "policy": [fixed_policy, linear_ts, beta_ts, regression_ts],
    "model_from_patient_id": [
        data_generating_model_1_1,
        data_generating_model_1_2,
        data_generating_model_1_3,
    ],
}
configurations = generate_configuration_cross_product(study_designs)

In [ ]:
calculated_series, config_to_simulation_data = simulate_configurations(
    configurations, LENGTH
)

In [ ]:
generate_result_table(calculated_series)

In [ ]:
plot_regret_against_fixed(calculated_series, config_to_simulation_data)

In [ ]:
def plot_allocations_for_calculated_series(calculated_series):
    panels = [series["result"].plot_allocations() for series in calculated_series]
    for panel, i in zip(panels, range(len(panels))):
        panel.opts(
            title=f"{calculated_series[i]['configuration']['policy']}, {calculated_series[i]['configuration']['model']}",
            fontsize={"title": "80%"},
        )
    return holoviews.Layout(panels).cols(1)


plot_allocations_for_calculated_series(calculated_series)

### Results
- Both models perform well: Note that -40 is the perfect regret for 1 as an intervention effect, and the models achieve roundabout half of that
- Conjugate Normal Model performs a little bit better on average, but has some higher variability

### Identified problems
- None

## Scenario 2: Baseline Changes
Most likely, our baseline will not be constant over time. Let's see how this affects the performance of our algorithms.
For this, we now simulate the baseline

- as a linear function
- as a gaussian process:
$c$ is the correlation coefficient, and $\sigma$ the variance of the process  
$y_t = c y_{t-1} + \epsilon$  
$\epsilon \sim N(0, \sigma)$  



In [ ]:
data_generating_model_2_1 = lambda patient_id: SelfExperimentationModel(
    patient_id,
    intervention_effects=[0, 1],
    baseline_model="linear",
    baseline_config={"start": 10, "end": 30, "min": 1, "max": -1, "name": "Ramp Down"},
)
data_generating_model_2_2 = lambda patient_id: SelfExperimentationModel(
    patient_id,
    intervention_effects=[0, 1],
    baseline_model="linear",
    baseline_config={"start": 10, "end": 30, "min": -1, "max": 1, "name": "Ramp Up"},
)
data_generating_model_2_3 = lambda patient_id: SelfExperimentationModel(
    patient_id,
    intervention_effects=[0, 1],
    baseline_model="random_walk",
    baseline_config={
        "min": -1,
        "max": 1,
        "variance": 0.1,
        "correlation": 0.99,
        "name": "RandomWalk1",
    },
)
data_generating_model_2_4 = lambda patient_id: SelfExperimentationModel(
    patient_id,
    intervention_effects=[0, 1],
    baseline_model="random_walk",
    baseline_config={
        "min": -2,
        "max": 2,
        "variance": 0.3,
        "correlation": 0.999,
        "name": "RandomWalk2",
    },
)
data_generating_model_2_5 = lambda patient_id: SelfExperimentationModel(
    patient_id,
    intervention_effects=[0, 0],
    baseline_model="linear",
    baseline_config={"start": 10, "end": 30, "min": 1, "max": -1, "name": "Ramp Down"},
)
data_generating_model_2_6 = lambda patient_id: SelfExperimentationModel(
    patient_id,
    intervention_effects=[0, 0],
    baseline_model="linear",
    baseline_config={"start": 10, "end": 30, "min": -1, "max": 1, "name": "Ramp Up"},
)

In [ ]:
study_designs = {
    "n_patients": [N_PATIENTS],
    "policy": [
        fixed_policy,
        linear_ts,
        beta_ts,
        ClippedThompsonSampling(
            inference_model=ConjugateNormalModel(**conjugate_normal_model_priors),
            number_of_actions=N_INTERVENTIONS,
        ),
    ],
    "model_from_patient_id": [
        data_generating_model_2_1,
        data_generating_model_2_2,
        data_generating_model_2_3,
        data_generating_model_2_4,
        data_generating_model_2_5,
        data_generating_model_2_6,
    ],
}
configurations = generate_configuration_cross_product(study_designs)

In [ ]:
# Takes some time
calculated_series, config_to_simulation_data = simulate_configurations(
    configurations, LENGTH
)

In [ ]:
# Look at some baselines:
baseline_names = ["Ramp Down", "Ramp Up", "RandomWalk1", "RandomWalk2"]


def print_baseline(baseline_name):
    for series in calculated_series:
        if baseline_name in series["configuration"]["model"]:
            series["result"].simulations[0].history.to_df().plot.line(
                x="t", y="baseline"
            )
            plt.title(baseline_name)
            return


for baseline_name in baseline_names:
    print_baseline(baseline_name)

In [ ]:
generate_result_table(calculated_series)

In [ ]:
plot_regret_against_fixed(calculated_series, config_to_simulation_data)

In [ ]:
plot_allocations_for_calculated_series(calculated_series)

### Results:
- Similar results on Regret for both models
- See "SelfExperimentationModel[[0, 1], Ramp Up, 0, 0]", "Beta Model":
  - Even though we have a positive treatment effect, ~20% of the cases converge on the baseline treatment
- See allocation plots for SEM[0, 0], Ramp Up
  - both models converge heavily towards one condition at around t=20, which is 50% good or bad.

### Identified problems:
- Models converge if there is no treatment effect
- Models converge on the wrong treatment effect (mostly present in Beta Model)

## Scenario 3: Correlated Outcomes
Most likely, our outcomes will be correlated over time. Let's see how this affects the performance of our algorithms.

For this we now simulate the outcomes as:
$outcome_t = correlationfactor \cdot outcome_{t-1} + \epsilon$


In [ ]:
data_generating_model_3_1 = lambda patient_id: SelfExperimentationModel(
    patient_id, intervention_effects=[0, 1], correlation=0.3
)
data_generating_model_3_2 = lambda patient_id: SelfExperimentationModel(
    patient_id, intervention_effects=[0, 1], correlation=-0.3
)
data_generating_model_3_3 = lambda patient_id: SelfExperimentationModel(
    patient_id, intervention_effects=[0, 1], correlation=0.7
)
data_generating_model_3_4 = lambda patient_id: SelfExperimentationModel(
    patient_id, intervention_effects=[0, 1], correlation=-0.7
)
data_generating_model_3_5 = lambda patient_id: SelfExperimentationModel(
    patient_id, intervention_effects=[0, 0], correlation=-0.7
)
data_generating_model_3_6 = lambda patient_id: SelfExperimentationModel(
    patient_id, intervention_effects=[0, 0], correlation=-0.7
)

In [ ]:
study_designs = {
    "n_patients": [N_PATIENTS],
    "policy": [fixed_policy, linear_ts, beta_ts],
    "model_from_patient_id": [
        data_generating_model_3_1,
        data_generating_model_3_2,
        data_generating_model_3_3,
        data_generating_model_3_4,
        data_generating_model_3_5,
        data_generating_model_3_6,
    ],
}
configurations = generate_configuration_cross_product(study_designs)

In [ ]:
# Takes some time
calculated_series, config_to_simulation_data = simulate_configurations(
    configurations, LENGTH
)

In [ ]:
# Let's plot an outcome to see it's correlation:
calculated_series[2]["result"].simulations[0].history.to_df().plot(
    x="t", y="continuous_outcome"
)

In [ ]:
generate_result_table(calculated_series)

In [ ]:
plot_regret_against_fixed(calculated_series, config_to_simulation_data)

In [ ]:
plot_allocations_for_calculated_series(calculated_series)

### Result 
- The results in regret differ, which should not be because the models perform better but because we change the simulation environment. No sever bad effects can be seen by the different correlation factors.

### Identified problems:
None

## Scenario 4: Random Spikes

The outcome in a Self-Experiment is likely not only influenced by the intervention, but by other unobserved events.
We simulate this by introducing spikes into the data. A spike is a datapoint where we set the outcome to 1 regardless of the sampled condition.
We simulate by using a probability $p_{spike}$ which says how likely a spike at a current datapoint is.

In [ ]:
data_generating_model_4_1 = lambda patient_id: SelfExperimentationModel(
    patient_id,
    intervention_effects=[0, 1],
    spike_probability=1.0 / 7,
)
data_generating_model_4_2 = lambda patient_id: SelfExperimentationModel(
    patient_id,
    intervention_effects=[0, 1],
    spike_probability=2.0 / 7,
)
data_generating_model_4_3 = lambda patient_id: SelfExperimentationModel(
    patient_id, intervention_effects=[0, 1], spike_probability=3.0 / 7
)
data_generating_model_4_4 = lambda patient_id: SelfExperimentationModel(
    patient_id, intervention_effects=[0, 0], spike_probability=2.0 / 7
)

In [ ]:
study_designs = {
    "n_patients": [N_PATIENTS],
    "policy": [fixed_policy, linear_ts, beta_ts],
    "model_from_patient_id": [
        data_generating_model_4_1,
        data_generating_model_4_2,
        data_generating_model_4_3,
        data_generating_model_4_4,
    ],
}
configurations = generate_configuration_cross_product(study_designs)

In [ ]:
calculated_series, config_to_simulation_data = simulate_configurations(
    configurations, LENGTH
)

In [ ]:
generate_result_table(calculated_series)

In [ ]:
plot_regret_against_fixed(calculated_series, config_to_simulation_data)

In [ ]:
plot_allocations_for_calculated_series(calculated_series)

### Results:
- Spike probability correlates linear with a drop in regret performance
- Spikes quickly lead to the model believing that there is one, and converging to the wrong treatment
  - Problem is less present in ConjugateNormalModel, after some data, it will converge back

### Identified problems:
- Spikes in data might lead to convergence to condition without treatment effect
- Spikes in data might lead to long phases with testing a poor condition

## Scenario 5: All together
All combinations of two effects and all three effects from Scenarios 2-4 together, to test if there are any interaction effects. I choose Ramp Up as the baseline change, cause this was doing the most harm.

In [ ]:
data_generating_model_5_1 = lambda patient_id: SelfExperimentationModel(
    patient_id,
    intervention_effects=[0, 1],
    baseline_model="linear",
    baseline_config={"start": 10, "end": 30, "min": -1, "max": 1, "name": "Ramp Up"},
    correlation=0.7,
)
data_generating_model_5_2 = lambda patient_id: SelfExperimentationModel(
    patient_id,
    intervention_effects=[0, 1],
    spike_probability=2.0 / 7,
    baseline_model="linear",
    baseline_config={"start": 10, "end": 30, "min": -1, "max": 1, "name": "Ramp Up"},
)
data_generating_model_5_3 = lambda patient_id: SelfExperimentationModel(
    patient_id,
    intervention_effects=[0, 1],
    spike_probability=2.0 / 7,
    correlation=0.7,
)
data_generating_model_5_4 = lambda patient_id: SelfExperimentationModel(
    patient_id,
    intervention_effects=[0, 1],
    spike_probability=2.0 / 7,
    baseline_model="linear",
    baseline_config={"start": 10, "end": 30, "min": -1, "max": 1, "name": "Ramp Up"},
    correlation=0.7,
)

In [ ]:
study_designs = {
    "n_patients": [N_PATIENTS],
    "policy": [fixed_policy, linear_ts, beta_ts],
    "model_from_patient_id": [
        data_generating_model_5_1,
        data_generating_model_5_2,
        data_generating_model_5_3,
        data_generating_model_5_4,
    ],
}
configurations = generate_configuration_cross_product(study_designs)

In [ ]:
calculated_series, config_to_simulation_data = simulate_configurations(
    configurations, LENGTH
)

In [ ]:
generate_result_table(calculated_series)

In [ ]:
plot_regret_against_fixed(calculated_series, config_to_simulation_data)

In [ ]:
plot_allocations_for_calculated_series(calculated_series)

### Results
- General decrease in performance in Regret
- Correlation & Spikes lead to high variability in outcome
- Ramp Up + X leads to big drop in performance
  - BetaModel performs the bad with RampUp and Correlation
  - Conjugate Model performs worst with all three influences
- Big "swing" at t~15 for BetaModel with RampUp and Correlation
  - Probably due to average changing through new observations and then sudden changes in calculated probabilities

### Identified problems:
- Correlation & Spikes lead to high variability in outcome
- Trend towards the wrong condition with different interfering factors

## Mitigation

### List of identified problems:
#### Scenario 2
- Models converge if there is no treatment effect
- Models converge on the wrong treatment effect (mostly present in Beta Model). 
#### Scenario 4
- Spikes in data might lead to convergence to condition without treatment effect
- Spikes in data might lead to long phases with testing a poor condition
#### Scenario 5
- Correlation & Spikes lead to high variability in outcome
- Trend towards the wrong condition with different interfering factors

### Mitigation strategies
- Clipping
Fixing the max probability for one condition, so that the model can not "endlessly converge".
- Sliding Window
Only analyzing x newest datapoints, so that old data is not incorporated.
- Starting Balanced
Starting the experiment with a fixed ABAB design, and only then starting ThompsonSampling.

### Test scenarios
We test against all inferences, once with no intervention effect and once with.

In [ ]:
fixed_policy = FixedPolicy(number_of_actions=N_INTERVENTIONS)
clipped_linear_ts = ClippedThompsonSampling(
    inference_model=ConjugateNormalModel(**conjugate_normal_model_priors),
    number_of_actions=N_INTERVENTIONS,
)
clipped_beta_ts = ClippedThompsonSampling(
    inference_model=BetaModel(),
    number_of_actions=N_INTERVENTIONS,
)
fixed_start_linear_ts = ComposedPolicy(
    policies=[
        FixedPolicy(number_of_actions=N_INTERVENTIONS),
        ThompsonSampling(
            inference_model=ConjugateNormalModel(**conjugate_normal_model_priors),
            number_of_actions=N_INTERVENTIONS,
        ),
    ],
    durations=[4, 100],
    number_of_actions=N_INTERVENTIONS,
)
fixed_start_beta_ts = ComposedPolicy(
    policies=[
        FixedPolicy(number_of_actions=N_INTERVENTIONS),
        ThompsonSampling(
            inference_model=BetaModel(),
            number_of_actions=N_INTERVENTIONS,
        ),
    ],
    durations=[4, 100],
    number_of_actions=N_INTERVENTIONS,
)

sliding_linear_ts = ThompsonSampling(
    inference_model=SlidingModel(
        sliding_window_size=15,
        model=ConjugateNormalModel(**conjugate_normal_model_priors),
    ),
    number_of_actions=N_INTERVENTIONS,
)

sliding_beta_ts = ThompsonSampling(
    inference_model=SlidingModel(sliding_window_size=15, model=BetaModel()),
    number_of_actions=N_INTERVENTIONS,
)

In [ ]:
data_generating_model_6_1 = lambda patient_id: SelfExperimentationModel(
    patient_id,
    intervention_effects=[0, 1],
    spike_probability=2.0 / 7,
    baseline_model="linear",
    baseline_config={"start": 10, "end": 30, "min": -1, "max": 1, "name": "Ramp Up"},
    correlation=0,
)
data_generating_model_6_2 = lambda patient_id: SelfExperimentationModel(
    patient_id,
    intervention_effects=[0, 0],
    spike_probability=2.0 / 7,
    baseline_model="linear",
    baseline_config={"start": 10, "end": 30, "min": -1, "max": 1, "name": "Ramp Up"},
    correlation=0,
)
data_generating_model_6_3 = lambda patient_id: SelfExperimentationModel(
    patient_id,
    intervention_effects=[0, 1],
    spike_probability=2.0 / 7,
    baseline_model="linear",
    baseline_config={"start": 10, "end": 30, "min": -1, "max": 1, "name": "Ramp Up"},
    correlation=0.7,
)

In [ ]:
study_designs = {
    "n_patients": [N_PATIENTS],
    "policy": [
        fixed_policy,
        linear_ts,
        beta_ts,
        clipped_linear_ts,
        clipped_beta_ts,
        fixed_start_beta_ts,
        fixed_start_linear_ts,
        sliding_linear_ts,
        sliding_beta_ts,
    ],
    "model_from_patient_id": [
        data_generating_model_6_1,
        data_generating_model_6_2,
        data_generating_model_6_3,
    ],
}
configurations = generate_configuration_cross_product(study_designs)

In [ ]:
calculated_series, config_to_simulation_data = simulate_configurations(
    configurations, LENGTH
)

In [ ]:
generate_result_table(calculated_series)

In [ ]:
plot_regret_against_fixed(calculated_series, config_to_simulation_data)

In [ ]:
plot_allocations_for_calculated_series(calculated_series)

### Results
- Clipping:
  - Without correlation: Improves prediction
  - With correlation: Only small effect: Prevents really long sequences, but end decision is not touched, (probably because due to correlation only switching shortly does not make a difference)
  - We see that it prevents always sampling the wrong condition (no "lock in")
- Sliding Window:
  - No clear benefits
- Balanced Start:
  -  Increases correct prediction at end of trial
  -  Increases variance (counter-intuitive)

### Combining what we learned

In [ ]:
### "The ultimate thompson sampling"
ultimate_ts = ComposedPolicy(
    policies=[
        FixedPolicy(number_of_actions=N_INTERVENTIONS),
        ClippedThompsonSampling(
            inference_model=ConjugateNormalModel(**conjugate_normal_model_priors),
            number_of_actions=N_INTERVENTIONS,
        ),
    ],
    durations=[4, 100],
    number_of_actions=N_INTERVENTIONS,
)
study_designs = {
    "n_patients": [N_PATIENTS],
    "policy": [fixed_policy, ultimate_ts, beta_ts],
    "model_from_patient_id": [
        data_generating_model_1_1,
        data_generating_model_1_2,
        data_generating_model_1_3,
        data_generating_model_2_1,
        data_generating_model_2_2,
        data_generating_model_2_3,
        data_generating_model_3_1,
        data_generating_model_3_2,
        data_generating_model_3_3,
        data_generating_model_4_1,
        data_generating_model_4_2,
        data_generating_model_4_3,
        data_generating_model_5_1,
        data_generating_model_5_2,
        data_generating_model_5_3,
        data_generating_model_6_1,
        data_generating_model_6_2,
        data_generating_model_6_3,
    ],
}
configurations = generate_configuration_cross_product(study_designs)
calculated_series, config_to_simulation_data = simulate_configurations(
    configurations, LENGTH
)

In [ ]:
generate_result_table(calculated_series)

In [ ]:
plot_allocations_for_calculated_series(calculated_series)

## Results
- model has similar performance in regret, but much better performance in pointing out the correct intervention


## Learnings
Behavior of Thomspons Sampling is different, heavily interactive between simulation model and chosen implementation.
E.g. clipping: One might think it would decrease the variability in performance, but it does not always in our experiments.

### New model
I propose implementing a model that randomizes to a balanced design for the first 4 datapoints, either ABAB or BABA, and then start with ClippedThompsonSampling with a normal model:
- Easy to update due to conjugate prior, which means we can use simple math and don't need to sample
- Easy to calculate probabilities, we only need to sample two distribution
- Is able to "understand" the magnitude of change in the data
#### Potential shortcomings:
- model assumes some "linearity" on the scale of results, which is not always given
- clipping will force some exploration even when the data is perfectly clear


### Open questions:

- Can such a model provide better user experience? --> Ultimately only testable in a user study, I think this could be interesting, but not sure if we want to go that route

- How can data labelling help? --> We discussed a lot if it would help if users could "label" data to be influenced not by the condition, but by something else, and are outliers. The first thing that comes to mind is a weighted linear regression, but it is not really clear how to transfer this into a Bayesian Approach (for a fun read, see Andrew Gelman discussing if weights are Bayesian and should be allowed in stan)

## Additional code